https://adventofcode.com/2023/day/5

In [1]:
import math
from collections import deque

In [2]:
with open("data/05.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

In [4]:
class GardenMap:
    def __init__(self, data):
        self.ranges = []
        self.name = "anon"
        for line in data.splitlines():
            if line and line[0].isdigit():
                self.ranges.append(tuple(int(x) for x in line.split()))
            elif line and line[0].isalpha():
                self.name = line.split()[0]

    def find_range(self, s):
        try:
            r = max((x for x in self.ranges if x[1] <= s), key=lambda y: y[1])
        except ValueError:
            return
        d0, s0, lim = r
        if s0 + lim > s:
            return r

    def lookup(self, key):
        r = self.find_range(key)
        if r is None:
            return key
        d0, s0, lim = r
        return key - s0 + d0

    def __repr__(self):
        return f"<GardenMap {self.name}>"

In [5]:
def parse_input(data):
    seedstr, mapstr = data.split("\n\n", 1)
    seeds = [int(x) for x in seedstr.split()[1:]]
    maps = [GardenMap(x) for x in mapstr.split("\n\n")]
    return seeds, maps

In [6]:
def min_location(data):
    seeds, maps = parse_input(data)
    minloc = math.inf
    for s in seeds:
        for map in maps:
            s = map.lookup(s)
        minloc = min(minloc, s)
    return minloc

In [7]:
min_location(testdata)

35

In [8]:
min_location(data)

178159714

### Part 2

In [9]:
def gardenmap_2(data):
    ranges = []
    for line in data.splitlines():
        if line and line[0].isdigit():
            d0, s0, rl = (int(x) for x in line.split())
            ranges.append({"start": s0, "stop": s0 + rl, "offset": d0 - s0})
    ranges.sort(key=lambda x: x["start"])
    return ranges

In [10]:
def parse_input_2(data):
    seedstr, mapstr = data.split("\n\n", 1)
    seeds = [int(x) for x in seedstr.split()[1:]]
    maps = [gardenmap_2(x) for x in mapstr.split("\n\n")]
    return seeds, maps

In [11]:
seeds, maps = parse_input_2(testdata)

In [12]:
seeds

[79, 14, 55, 13]

In [13]:
seeds[:-1:2]

[79, 55]

In [14]:
seeds[1::2]

[14, 13]

In [15]:
zip(seeds[:])

In [16]:
maps

[[{'start': 50, 'stop': 98, 'offset': 2},
  {'start': 98, 'stop': 100, 'offset': -48}],
 [{'start': 0, 'stop': 15, 'offset': 39},
  {'start': 15, 'stop': 52, 'offset': -15},
  {'start': 52, 'stop': 54, 'offset': -15}],
 [{'start': 0, 'stop': 7, 'offset': 42},
  {'start': 7, 'stop': 11, 'offset': 50},
  {'start': 11, 'stop': 53, 'offset': -11},
  {'start': 53, 'stop': 61, 'offset': -4}],
 [{'start': 18, 'stop': 25, 'offset': 70},
  {'start': 25, 'stop': 95, 'offset': -7}],
 [{'start': 45, 'stop': 64, 'offset': 36},
  {'start': 64, 'stop': 77, 'offset': 4},
  {'start': 77, 'stop': 100, 'offset': -32}],
 [{'start': 0, 'stop': 69, 'offset': 1},
  {'start': 69, 'stop': 70, 'offset': -69}],
 [{'start': 56, 'stop': 93, 'offset': 4},
  {'start': 93, 'stop': 97, 'offset': -37}]]

In [31]:
def min_location_2(data):
    seeds, gmaps = parse_input_2(data)
    q = deque([])
    minloc = math.inf
    for start, lim in zip(seeds[:-1:2], seeds[1::2]):
        q.append({"start": start, "stop": start + lim, "nextmap": 0})
    while q:
        r1 = q.popleft()
        try:
            gmap = gmaps[r1["nextmap"]]
        except ValueError:
            minloc = min(minloc, r1["start"])
            continue
        q.extend(reachable_ranges(r1, gmap))
    return minloc


def reachable_ranges(r, gmap):
    q = deque([r])
    while q:
        r = q.popleft()
        if r["start"] >= r["stop"]:
            continue
        for target in gmap:
            if is_overlap(r, target):
                break
        else:
            y = {"start": r["start"], "stop": r["stop"], "nextmap": r["nextmap"] + 1}
            print(y)
            yield y
            continue
        newstart = max(r["start"], target["start"])
        newstop = min(r["stop"], target["stop"])
        offset = target["offset"]
        y = {"start": newstart + offset, "stop": newstop + offset, "nextmap": r["nextmap"] + 1}
        # print(y)
        yield y
        q.append({"start": r["start"], "stop": newstart, "nextmap": r["nextmap"]})
        q.append({"start": newstop, "stop": r["stop"], "nextmap": r["nextmap"]})


def is_overlap(a, b):
    return not (
        a["stop"] < b["start"]
        or a["start"] > b["stop"]
        or b["stop"] < a["start"]
        or b["start"] > a["stop"]
    )

In [32]:
min_location_2(testdata)

{'start': 81, 'stop': 95, 'nextmap': 2}
{'start': 57, 'stop': 70, 'nextmap': 2}
{'start': 81, 'stop': 95, 'nextmap': 3}


KeyboardInterrupt: 